In [ ]:
import re

# ===== 定数 =====
WEEK_EN = ["SUN","MON","TUE","WED","THU","FRI","SAT"]
WEEK_JA = ["日","月","火","水","木","金","土"]
WEEK_TO_NUM = {name:i for i, name in enumerate(WEEK_EN)}
MONTH_EN = ["JAN","FEB","MAR","APR","MAY","JUN","JUL","AUG","SEP","OCT","NOV","DEC"]
MONTH_TO_NUM = {name:i+1 for i, name in enumerate(MONTH_EN)}

def _pad2(n:int) -> str:
    return f"{int(n):02d}"

# ===== 前処理 & 正規化（5フィールドへ） =====
def _unwrap(expr: str) -> str:
    s = expr.strip().replace("\u3000", " ")
    m = re.fullmatch(r"(?is)\s*cron\(\s*(.*?)\s*\)\s*", s)
    if m:
        s = m.group(1)
    return re.sub(r"\s+", " ", s).strip()

def _normalize_to_5(expr: str):
    """
    5 / 6 / 7 フィールドを 5フィールド (min hour dom mon dow) に正規化。
    - 6フィールド: 末尾が年なら落とす（AWS）／先頭が秒なら落とす（Quartz）
    - 7フィールド: 先頭の秒と末尾の年を落とす（Quartz）
    - '?' は '*' として扱う
    返り値: (minute, hour, dom, mon, dow, year_or_none, sec_or_none)
    """
    s = _unwrap(expr)
    parts = s.split()
    parts = [("*" if p == "?" else p).upper() for p in parts]

    if len(parts) == 5:
        sec, year = None, None
        minute, hour, dom, mon, dow = parts
    elif len(parts) == 6:
        # 年の判定（*, 2025, 2020-2030, */2, 2020/2, 2020,2022,... を許容）
        last = parts[-1]
        is_yearish = re.fullmatch(r"\*|(\d{4})(?:-\d{4})?(?:/\d+)?(?:,\d{4})*", last) is not None
        if is_yearish:
            sec = None
            minute, hour, dom, mon, dow, year = parts
        else:
            sec, minute, hour, dom, mon, dow = parts
            year = None
    elif len(parts) == 7:
        sec, minute, hour, dom, mon, dow, year = parts
    else:
        raise ValueError(f"未対応のフィールド数: {len(parts)} -> {parts}")

    return (minute, hour, dom, mon, dow, year, sec)

# ===== ユーティリティ（数値/英名処理） =====
def _num_or_name_to_num(tok: str, kind: str) -> int:
    t = tok.strip().upper()
    if kind == "dow":
        if t in WEEK_TO_NUM: return WEEK_TO_NUM[t]
        if t == "7": return 0  # 0/7=Sun
    if kind == "mon":
        if t in MONTH_TO_NUM: return MONTH_TO_NUM[t]
    return int(t)

def _piece_list(expr: str) -> list[str]:
    return [p for p in expr.split(",")]

def _range_to_jp(a: str, b: str, unit: str, kind: str|None=None) -> str:
    if kind == "dow":
        a_n = _num_or_name_to_num(a, "dow"); b_n = _num_or_name_to_num(b, "dow")
        return f"{WEEK_JA[a_n]}〜{WEEK_JA[b_n]}"
    if kind == "mon":
        a_n = _num_or_name_to_num(a, "mon"); b_n = _num_or_name_to_num(b, "mon")
        return f"{a_n}月〜{b_n}月"
    return f"{int(a)}〜{int(b)}{unit}"

def _step_to_jp(base: str, step: str, unit: str) -> str:
    if base == "*":
        return f"{int(step)}{unit}ごと"
    if "-" in base:
        a, b = base.split("-")
        return f"{int(a)}〜{int(b)}{unit}の{int(step)}{unit}間隔"
    return f"{int(base)}{unit}から{int(step)}{unit}ごと"

def _list_to_jp(pieces: list[str], unit: str, kind: str|None=None) -> str:
    out = []
    for p in pieces:
        if "/" in p:
            base, st = p.split("/")
            out.append(_step_to_jp(base, st, unit))
        elif "-" in p:
            a, b = p.split("-")
            out.append(_range_to_jp(a, b, unit, kind))
        elif kind == "dow":
            n = _num_or_name_to_num(p, "dow"); out.append(WEEK_JA[n])
        elif kind == "mon":
            n = _num_or_name_to_num(p, "mon"); out.append(f"{n}月")
        else:
            out.append(f"{int(p)}{unit}")
    return "・".join(out)

# ===== DOM / DOW / MON 日本語化（Quartzの主要拡張に対応） =====
def _dom_to_jp(dom: str) -> str:
    dom = dom.upper()
    if dom == "*": return "毎日"
    if dom == "L": return "毎月の月末日"
    if dom in ("LW", "WL"): return "毎月の月末平日"
    m = re.fullmatch(r"(\d+)W", dom)
    if m: return f"毎月{int(m.group(1))}日に最も近い平日"

    if "," in dom: return f"毎月{_list_to_jp(_piece_list(dom), '日')}"
    if "/" in dom:
        base, st = dom.split("/")
        if base == "*":
            base_jp = "1〜31日"
        elif "-" in base:
            a, b = base.split("-"); base_jp = f"{int(a)}〜{int(b)}日"
        else:
            base_jp = f"{int(base)}日"
        return f"毎月{base_jp}の{int(st)}日おき"
    if "-" in dom:
        a, b = dom.split("-"); return f"毎月{int(a)}〜{int(b)}日"
    return f"毎月{int(dom)}日"

def _dow_to_jp(dow: str) -> str:
    d = dow.upper()
    if d == "*": return "毎週すべての曜日"
    if d in ("1-5","MON-FRI"): return "平日"
    if d in ("0,6","6,0","SUN,SAT","SAT,SUN"): return "土日"

    m = re.fullmatch(r"([0-7A-Z]{3})L", d)
    if m:
        n = _num_or_name_to_num(m.group(1), "dow")
        return f"毎月の最後の{WEEK_JA[n]}曜日"
    m = re.fullmatch(r"([0-7A-Z]{3})#([1-5])", d)
    if m:
        n = _num_or_name_to_num(m.group(1), "dow")
        k = int(m.group(2))
        return f"毎月第{k}{WEEK_JA[n]}曜日"
    if d == "L":
        return "毎月の最後の週（Quartz L）"

    if "," in d: return "毎週" + _list_to_jp(_piece_list(d), "曜", kind="dow")
    if "/" in d:
        base, st = d.split("/")
        base_jp = "全曜日" if base == "*" else _range_to_jp(*base.split("-"), "曜", "dow") if "-" in base else WEEK_JA[_num_or_name_to_num(base,"dow")]
        return f"毎週{base_jp}の{int(st)}週ごと"
    if "-" in d:
        a, b = d.split("-"); return "毎週" + _range_to_jp(a, b, "曜", "dow")
    n = _num_or_name_to_num(d, "dow"); return f"毎週{WEEK_JA[n]}曜日"

def _mon_to_jp(mon: str) -> str:
    m = mon.upper()
    if m == "*": return "毎年"
    if "," in m: return "毎年" + _list_to_jp(_piece_list(m), "", kind="mon")
    if "/" in m:
        base, st = m.split("/")
        if base == "*": return f"{int(st)}か月ごと"
        if "-" in base:
            a, b = base.split("-"); return f"{_range_to_jp(a, b, '月', 'mon')}の{int(st)}か月ごと"
        return f"{_num_or_name_to_num(base,'mon')}月から{int(st)}か月ごと"
    if "-" in m:
        a, b = m.split("-"); return _range_to_jp(a, b, "月", "mon")
    return f"毎年{_num_or_name_to_num(m,'mon')}月"

def _year_to_jp(year: str|None) -> str:
    if not year or year == "*": return ""
    y = year.upper()
    if "," in y:
        ys = [int(x) for x in y.split(",")]
        return "（対象年: " + "・".join(f"{v}年" for v in ys) + "）"
    if "/" in y:
        base, st = y.split("/")
        if base == "*": return f"（対象年: {int(st)}年ごと）"
        if "-" in base:
            a, b = base.split("-"); return f"（対象年: {int(a)}〜{int(b)}年の{int(st)}年ごと）"
        return f"（対象年: {int(base)}年から{int(st)}年ごと）"
    if "-" in y:
        a, b = y.split("-"); return f"（対象年: {int(a)}〜{int(b)}年）"
    return f"（対象年: {int(y)}年）"

# ===== 分・時 を 1日の時刻例（HH:MM, …）へ =====
def _expand_num_field(expr: str, minv: int, maxv: int, field_name: str) -> list[int]:
    vals: set[int] = set()
    for tok in expr.split(","):
        tok = tok.strip()
        if tok == "*":
            vals.update(range(minv, maxv + 1)); continue
        if "/" in tok:
            base, step_str = tok.split("/", 1)
            if not step_str.isdigit(): raise ValueError(f"{field_name}: ステップが数値ではありません: {tok}")
            step = int(step_str)
            if step <= 0: raise ValueError(f"{field_name}: ステップは1以上にしてください: {tok}")
            if base == "*":
                start, end = minv, maxv
            elif "-" in base:
                a, b = map(int, base.split("-", 1))
                if a > b: raise ValueError(f"{field_name}: レンジが逆順です: {tok}")
                if a < minv or b > maxv: raise ValueError(f"{field_name}: レンジが範囲外です: {tok}")
                start, end = a, b
            else:
                a = int(base)
                if a < minv or a > maxv: raise ValueError(f"{field_name}: 値が範囲外です: {tok}")
                start, end = a, maxv
            v = start
            while v <= end:
                vals.add(v); v += step
            continue
        if "-" in tok:
            a, b = map(int, tok.split("-", 1))
            if a > b: raise ValueError(f"{field_name}: レンジが逆順です: {tok}")
            if a < minv or b > maxv: raise ValueError(f"{field_name}: レンジが範囲外です: {tok}")
            vals.update(range(a, b + 1)); continue
        v = int(tok)
        if v < minv or v > maxv: raise ValueError(f"{field_name}: 値が範囲外です: {tok}")
        vals.add(v)
    if not vals:
        raise ValueError(f"{field_name}: 値が解決できません: {expr!r}")
    return sorted(vals)

def _times_examples(min_expr: str, hour_expr: str, max_items: int = 48) -> str:
    hours = _expand_num_field(hour_expr, 0, 23, "時")
    mins  = _expand_num_field(min_expr,  0, 59, "分")
    times = [f"{_pad2(h)}:{_pad2(m)}" for h in hours for m in mins]
    times.sort()
    return ",".join(times[:max_items]) + (",..." if len(times) > max_items else "")

# ===== メイン =====
def cron_to_japanese_with_examples(expr: str, max_times: int = 48) -> str:
    """
    ・5/6/7フィールド対応（秒と年は自動で捨てる/注記）
    ・分・時は HH:MM,HH:MM,... を列挙（多い場合は ... で省略）
    ・日/月/曜日は日本語に整形
    ・JST想定（時差計算なし）
    """
    minute, hour, dom, mon, dow, year, sec = _normalize_to_5(expr)

    # 月・日・曜日（DOMとDOWが同時指定なら OR 表現）
    mon_jp = _mon_to_jp(mon)
    dom_used, dow_used = (dom != "*"), (dow != "*")
    if dom_used and dow_used:
        mid = f"{_dom_to_jp(dom).replace('毎月','')} または {_dow_to_jp(dow)}"
    elif dom_used:
        mid = _dom_to_jp(dom)
    elif dow_used:
        mid = _dow_to_jp(dow)
    else:
        mid = "毎日"

    prefix = "" if mon_jp == "毎年" else (mon_jp + " ")
    times_example = _times_examples(minute, hour, max_items=max_times)
    year_note = _year_to_jp(year)

    return f"{prefix}{mid} {times_example}{year_note}（JST）"
